In [ ]:
!nvidia-smi

In [ ]:
!pip install -U keras-efficientnet-v2
!pip install tensorflow_addons

In [ ]:
#%%
# my simple example of sorghum classification. It uses original "sorghum-id-fgvc-9" images to train&inference, so it's very slow. I recommend use tfrecord dataset like https://www.kaggle.com/code/tchaye59/512x512-images-to-tfrecords
# It's just sample, so you should tune the parameters for more score!


import tensorflow as tf
import keras_efficientnet_v2
import pandas as pd
from sklearn.model_selection import train_test_split
import os

proj_dir = "/kaggle/input/sorghum-id-fgvc-9/"
output_dir = "output/"

df = pd.read_csv(proj_dir + "train_cultivar_mapping.csv")

base_path = proj_dir + "train_images/"

df["fullpath"] = base_path + df["image"] 

# omit NA images
exists = []

for i in df["fullpath"]:
    if not os.path.exists(i):
        exists.append(False)
        print(i)
    else:
        exists.append(True)

df["exist"] = pd.Series(exists)
df = df[df.exist]
print("available images: " + str(len(df)))

In [ ]:
BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE
TRAIN_SIZE = 0.95
SHUFFLE_SIZE = 3000
test_data_paths = proj_dir + 'test/*.png'
WIDTH = 256
HEIGHT = 256
EPOCHS = 1
MAGNITUDE = 5
model_name = "simple_effnet"

paths = df["fullpath"]
labels_str = df["cultivar"]


image_count = len(paths)
image_count

In [ ]:
label_to_index = dict((name, index) for index,name in enumerate(labels_str.unique()))
label_to_index

labels_idx = labels_str.map(lambda x: label_to_index[x])
labels_idx

In [ ]:
train_paths, val_paths, train_labels, val_labels = train_test_split(paths, labels_idx, train_size=TRAIN_SIZE, shuffle=True, random_state=42, stratify=labels_idx)

In [ ]:
def preprocess_image(path):
    image = tf.image.decode_png(path, channels=3)
    image = tf.image.resize(image, size=(HEIGHT,WIDTH))

    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label



In [ ]:
# define RandAug class(from keras_efficientnet_v2 sample codes)
# it's too slow, so commented out

class RandomProcessImage:
    def __init__(self, target_shape=(300, 300), magnitude=0, keep_shape=False):
        self.target_shape, self.magnitude, self.keep_shape = target_shape, magnitude, keep_shape
        self.target_shape = target_shape if len(target_shape) == 2 else target_shape[:2]
        if magnitude > 0:
            from keras_efficientnet_v2 import augment

            translate_const, cutout_const = 100, 40
            # translate_const = int(target_shape[0] * 10 / magnitude)
            # cutout_const = int(target_shape[0] * 40 / 224)
            print(">>>> RandAugment: magnitude = %d, translate_const = %d, cutout_const = %d" % (magnitude, translate_const, cutout_const))
            aa = augment.RandAugment(num_layers = 2, magnitude=magnitude, translate_const=translate_const, cutout_const=cutout_const)
            # aa.available_ops = ["AutoContrast", "Equalize", "Invert", "Rotate", "Posterize", "Solarize", "Color", "Contrast", "Brightness", "Sharpness", "ShearX", "ShearY", "TranslateX", "TranslateY", "Cutout", "SolarizeAdd"]
            # aa.available_ops = ["AutoContrast", "Equalize", "Invert", "Rotate", "Posterize", "Solarize", "Color", "Contrast", "Brightness", "Sharpness", "TranslateX", "TranslateY", "SolarizeAdd"]
            self.process = lambda img: aa.distort(img)
        elif magnitude == 0:
            self.process = lambda img: tf.image.random_flip_left_right(img)
        else:
            self.process = lambda img: img

    def __call__(self, datapoint):
        image = datapoint
        if self.keep_shape:
            cropped_shape = tf.reduce_min(tf.keras.backend.shape(image)[:2])
            image = tf.image.random_crop(image, (cropped_shape, cropped_shape, 3))

        input_image = tf.image.resize(image, self.target_shape)
        input_image = self.process(input_image)
        # input_image = (tf.cast(input_image, tf.float32) - 127.5) / 128
        return input_image

data_aug = RandomProcessImage((WIDTH, HEIGHT), MAGNITUDE, keep_shape=True)



In [ ]:
train_path_label_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_path_label_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

train_image_label_ds = train_path_label_ds.map(load_and_preprocess_from_path_label,num_parallel_calls=AUTOTUNE)
val_image_label_ds = val_path_label_ds.map(load_and_preprocess_from_path_label,num_parallel_calls=AUTOTUNE)

train_ds = train_image_label_ds
train_ds = train_ds.repeat()
train_ds = train_ds.shuffle(buffer_size=SHUFFLE_SIZE)
train_ds = train_ds.map(lambda x, y:(data_aug(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
train_ds

val_ds = val_image_label_ds.cache()
val_ds = val_ds.batch(BATCH_SIZE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
val_ds

In [ ]:
import matplotlib.pyplot as plt
def show_image_batch(images: list):
    """
    Displays a batch of image present in images
    """
    fig = plt.figure(figsize=(10,5))
    for idx in range(6):
        ax = plt.subplot(2, 3, idx+1)
        plt.imshow(images[idx])
        plt.axis("off")

def show_dataset(dataset):
    batch = next(iter(dataset))
    images, labels = batch

    plt.figure(figsize=(10, 10))
    for idx in range(9):
        ax = plt.subplot(3, 3, idx + 1)
        plt.imshow(images[idx].numpy().astype("uint8"))
        plt.title("Class: {}".format(labels[idx].numpy()))
        plt.axis("off")

show_dataset(train_ds)

In [ ]:
#Exclude model top layers by set num_classes=0
basemodel = keras_efficientnet_v2.EfficientNetV2B0(input_shape=(WIDTH, HEIGHT, 3), drop_connect_rate=0.2, num_classes=0, include_preprocessing=True, pretrained="imagenet")

# for this dataset, imagenet weights base `basemodel.trainable=False` is not suitable!
basemodel.trainable = True

image_input = tf.keras.layers.Input(shape=(WIDTH,HEIGHT,3))
out = basemodel(image_input)
out = tf.keras.layers.GlobalAveragePooling2D()(out)
out = tf.keras.layers.Dropout(0.2)(out)
out = tf.keras.layers.Dense(len(label_to_index), activation="softmax")(out)

model = tf.keras.Model(image_input, out)

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])


#%%
#thanks for https://www.kaggle.com/code/tchaye59/efficientnet-tensorflow-baseline-tpu
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy',mode='max', patience=20)
ckp_callback = tf.keras.callbacks.ModelCheckpoint(
                                            filepath=f'model.h5',
                                            save_weights_only=True,
                                            monitor='accuracy',
                                            mode='max',
                                            options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'),
                                            save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',mode='max',factor=0.2,patience=3, min_lr=1e-5)
callbacks=[callback,ckp_callback,reduce_lr]


#%%
model.summary()

#%%
steps_per_epoch=tf.math.ceil(len(train_paths)/BATCH_SIZE).numpy()
steps_per_epoch

In [ ]:
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, steps_per_epoch=steps_per_epoch, callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
import glob
test_image_paths = list(glob.glob(test_data_paths))
test_image_paths = [str(path) for path in test_image_paths]
len(test_image_paths)
#%%
path_label_test = tf.data.Dataset.from_tensor_slices(test_image_paths)

In [ ]:
test_ds = path_label_test.map(load_and_preprocess_image, tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(buffer_size=32)
#%%
predictions = model.predict(test_ds, batch_size=BATCH_SIZE, verbose=1)


In [ ]:
from tqdm import tqdm
prediction_output = [("filename", "cultivar")]

for i in tqdm(range(len(predictions.argmax(axis=1)))):
    pred_idx = predictions.argmax(axis=1)[i]
    prediction_output.append((os.path.basename(test_image_paths[i]), [k for k, v in label_to_index.items() if v == pred_idx][0]))
#%%
pd.DataFrame(prediction_output).to_csv("submission.csv", index=None, header=None)